In [204]:
import pandas as pd
pd.options.display.max_columns = 99

Read in the first five lines from loans_2007.csv and look for any data quality issues.

In [205]:
loans_2007df = pd.read_csv("loans_2007.csv")
print(loans_2007df.head(n=5))


        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501  1296599.0     5000.0       5000.0           4975.0   36 months   
1  1077430  1314167.0     2500.0       2500.0           2500.0   60 months   
2  1077175  1313524.0     2400.0       2400.0           2400.0   36 months   
3  1076863  1277178.0    10000.0      10000.0          10000.0   36 months   
4  1075358  1311748.0     3000.0       3000.0           3000.0   60 months   

  int_rate  installment grade sub_grade                 emp_title emp_length  \
0   10.65%       162.87     B        B2                       NaN  10+ years   
1   15.27%        59.83     C        C4                     Ryder   < 1 year   
2   15.96%        84.33     C        C5                       NaN  10+ years   
3   13.49%       339.31     C        C1       AIR RESOURCES BOARD  10+ years   
4   12.69%        67.79     B        B5  University Medical Group     1 year   

  home_ownership  annual_inc verification_status  

/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Read in the first 1000 rows from the data set, and calculate the total memory usage for these rows. Increase or decrease the number of rows to converge on a memory usage under five megabytes (to stay on the conservative side).

In [206]:
memory_usage_under_5mbs = loans_2007df[:3171].memory_usage(deep=True).sum() / (1024 ** 2)
print("Memory usage under five mbs and 3171 rows:\n")
print(memory_usage_under_5mbs)

Memory usage under five mbs and 3171 rows:

4.998802185058594


Let's familiarize ourselves with the columns to see which ones we can optimize. In the first mission, we explored column types by reading in the full dataframe. In this guided project, let's try to understand the column types better while using dataframe chunks.

For each chunk:
How many columns have a numeric type? How many have a string type?
How many unique values are there in each string column? 



In [207]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_float_dtype

chunk_iter = pd.read_csv("loans_2007.csv",chunksize=3170)
chunk_usage = []
for chunk in chunk_iter:
    chunk_usage.append(chunk.memory_usage())
    column_types = chunk.dtypes.value_counts()
    break
numerics = column_types[0] + column_types[2]
strings = column_types[1]

print("Total amount of numeric columns:\n")
print(numerics)
print('\n')
print("Total amount of string columns:\n")
print(strings)
    

Total amount of numeric columns:

31


Total amount of string columns:

21


How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?

In [208]:
#How many unique values are there in each string column? 
string_columns = loans_2007df.select_dtypes(include=['object'])
total_unique_values = len(string_columns)
print("Total amount of unique values:")
print(total_unique_values)
print('\n')
#How many of the string columns contain values that are less than 50% unique?
more_than_half_unique = 0
name_of_columns_more_than_half_the_vals_unique = []
for col in string_columns.columns:
    string_unique_values = len(loans_2007df[col].unique())
    string_total_values = len(loans_2007df[col])
    if string_unique_values / string_total_values > 0.5:
        more_than_half_unique += 1
        name_of_columns_more_than_half_the_vals_unique.append(col)
print('Strings columns where over half of the values where unique:')
print(more_than_half_unique)  


Total amount of unique values:
42538


Strings columns where over half of the values where unique:
2


Which float columns have no missing values and could be candidates for conversion to the integer type?

In [209]:
float_types = ["float16", "float32", "float64", "float128"]
float_columns = loans_2007df.select_dtypes(include= float_types)
missing_value_columns = float_columns.columns[float_columns.isnull().any()].tolist()
candidate_columns = []
column_names = list(loans_2007df.columns.values)
len(column_names)
for name in column_names:
    if name not in missing_value_columns:
        candidate_columns.append(name)
print('All candidate columns for conversion to integer type:\n')
print(candidate_columns)

All candidate columns for conversion to integer type:

['id', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type']


In [210]:
# Calculate the total memory usage across all of the chunks.
all_cols_memory = loans_2007df.memory_usage(deep = True)
all_cols_sum = all_cols_memory.sum()
print("Total memory usage:")
print(all_cols_sum / (1024*1024))

Total memory usage:
67.41611766815186


As we learned in the first mission of this course, we can achieve the greatest memory improvements by converting the string columns to a numeric type. Let's convert all of the columns where the values are less than 50% unique to the category type, and the columns that contain numeric values to the float type.

In [211]:
import numpy as np
numbers = [0,1,2,3,4,5,6,7,8,9]
can_be_numerics = []
can_be_categories = []
obj_cols = loans_2007df.select_dtypes(include=['object'])
for col in obj_cols.columns:
    for number in numbers:
        if number in loans_2007df[col]:
            can_be_numerics.append(col)
    num_unique_values = len(loans_2007df[col].unique())
    num_total_values = len(loans_2007df[col])
    if np.issubdtype(loans_2007df[col].dtype, np.number):
        loans_2007df[col] = loans_2007df[col].astype('float')
    elif num_unique_values / num_total_values < 0.5:
        can_be_categories.append(col)
        loans_2007df[col] = loans_2007df[col].astype('category')

While working with dataframe chunks:

In [212]:
#Determine which string columns you can convert to a numeric type if you 
#clean them. For example, the int_rate column is only a string because 
#of the % sign at the end.
print("String columns which can be numeric columns:\n")
print(set(can_be_numerics))
print('\n')            
#Determine which columns have a few unique values and convert them 
#to the category type.
print("Columns with few unique values:\n")
print(can_be_categories)



String columns which can be numeric columns:

{'term', 'home_ownership', 'revol_util', 'grade', 'issue_d', 'addr_state', 'sub_grade', 'emp_title', 'earliest_cr_line', 'loan_status', 'initial_list_status', 'emp_length', 'application_type', 'last_pymnt_d', 'last_credit_pull_d', 'int_rate', 'id', 'pymnt_plan', 'title', 'zip_code', 'purpose', 'verification_status'}


Columns with few unique values:

['term', 'int_rate', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type']


In [227]:
#Identify float columns that contain missing values, and that we 
#can convert to a more space efficient subtype
print("Float columns that contain missing values:\n")
print(missing_value_columns)
print("\n")
#Identify float columns that don't contain any missing values, 
#and that we can convert to the integer type because they represent 
#whole numbers
print("Float columns that don't contain any missing values\
 and can be represented as whole numbers:\n")
print(candidate_columns)

#Based on your conclusions, perform the necessary type changes across
#all chunks. Calculate the total memory footprint and compare it with
#the previous one.
new_memory_footprint = loans_2007df.memory_usage(deep = True).sum() / (1024*1024)
old_memory_footprint = all_cols_sum / (1024*1024)
print(all_cols_sum)
print(new_memory_footprint)
if all_cols_sum < new_memory_footprint:
    print("The new memory footprint is bigger at: " + str(new_memory_footprint )+ 
         " while the old memory footprint is: " + str(all_cols_sum / (1024*1024))) 
else:
    print("The old memory footprint is bigger at: " + str(all_cols_sum / (1024*1024)) + " megabytes " + 
         " while the new memory footprint is: " + str(new_memory_footprint) + " megabytes") 

Float columns that contain missing values:

['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens']


Float columns that don't contain any missing values and can be represented as whole numbers:

['id', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type']
70690923
17.914603233337402
The old me